# High-Level Setup & Imports

In [1]:
# General Python Packages
import os, time, numbers, math

# Torch Packages
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.optim import lr_scheduler, SGD
from torch.autograd import Variable
from torch import nn
from torch.nn import DataParallel
from torch.nn import Module

# General Analytics Packages
import pandas as pd
import numpy as np

# Visualization / Image Packages
import matplotlib.pyplot as plt
from PIL import Image

# Randomization Functions
from random import random as randuni

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:279: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
# Put MatPlotLib in interactive mode
plt.ion()

# Plot graphics inline in the notebook
%matplotlib inline

# Define Classes and Functions

### Image Data Utility Classes

In [3]:
def is_image_file(fname):
    """Checks if a file is an image.
    Args:
        fname (string): path to a file
    Returns:
        bool: True if the filename ends with a known image extension
    """
    return fname.lower().endswith('.png')

def create_label_maps(details_df):
    """ Take a descriptive dataframe and extract the unique labels and map to index values
    Args:
        details_df: Dataframe with the image details
    Returns:
        label_list: list of unique labels in the dataframe
        label_to_index: map from labels to indices
    """
    """ TODO: Research paper also excludes these labels but need to figure out how to handle
              cases that have these as positive findings (completely exclude?)
    excluded_labels = ['Edema','Hernia','Emphysema','Fibrosis','No Finding'
                      'Pleural_Thickening','Consolidation']
    """
    excluded_labels = ['No Finding']
    
    label_groups = details_df['Finding Labels'].unique()
    unique_labels = set([label for sublist in label_groups.tolist() for label in sublist.split('|')])
    
    # Drop some label that we do not want to include
    unique_labels = [l for l in unique_labels if l not in excluded_labels]

    index_to_label = {idx: val for idx, val in enumerate(unique_labels)}
    label_to_index = {val: idx for idx, val in index_to_label.items()}

    label_list = list(label_to_index.keys())

    return label_list, label_to_index

def create_image_list(dir):
    """ Create a full list of images available 
    Args:
        dir (string): root directory of images with subdirectories underneath
                      that have the .png images within them
    Returns:
        image_list: list of tuples with (image_name, full_image_path)
    """
    image_list = []
    dir = os.path.expanduser(dir)
    for subfolder in sorted(os.listdir(dir)):
        d = os.path.join(dir, subfolder)
        if not os.path.isdir(d):
            continue
        for subfolder_path, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                if is_image_file(fname):
                    path = os.path.join(subfolder_path, fname)
                    image_list.append((fname, path))
    return image_list

def pil_loader(path):
    """ Opens path as file with Pillow (https://github.com/python-pillow/Pillow/issues/835)
    Args:
        path (string): File path to the image
    Returns:
        img: Image in RGB format
    """
    f = open(path, 'rb')
    return Image.open(f)
    #with open(path, 'rb') as f:
    #    return Image.open(f)
        #with Image.open(f) as img:
        #    return img.load()
        
def imshow(inp, title=None):
    """ Convert tensor array to an image (only use post-dataset transform) """
    inp = inp[0]
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

### Torch Dataset Definition

In [4]:
class XrayImageSet(Dataset):
    """
    Args:
        image_root (string): root directory of the images in form image/subfolder/*.png
        csv_file (string): path to the CSV data file
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.
     Attributes:
        labels (list): list of the possible label names.
        label_to_index (dict): look from label name to a label index
        imgs (list): List of (filename, image path) tuples
    """
    
    def __init__(self, image_root, csv_file, transform=None, target_transform=None, loader = pil_loader):
        """ Create an instance of the Xray Dataset """
        img_details = pd.read_csv(csv_file)
        
        labels, label_to_index = create_label_maps(img_details)
        imgs = create_image_list(image_root)

        self.imgs = imgs
        self.image_details = img_details
        self.image_root = image_root
        self.labels = labels
        self.label_to_index = label_to_index
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        self.max_label_index = max(label_to_index.values())

    def __getitem__(self, index):
        """ Get image,labels pair by index
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        fname, path = self.imgs[index]
        target = self.get_one_hot_labels(fname)
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        """ Calculate length of the dataset (number of images) """
        return len(self.imgs)
    
    def get_labels(self, fname):
        """ Return the label string for the file """
        return self.image_details[self.image_details['Image Index'] == fname]['Finding Labels'].values[0]
    
    def one_hot_labels(self, labels):
        """ Convert the labels string (with each label separated by |) into 1-hot encoding """
        if labels == None:
            return None
        
        split_label_indices = [self.label_to_index.get(label)
                               for label in labels.split('|')
                               if label != 'No Finding']
        
        out = [1 if idx in split_label_indices else 0 for idx in range(self.max_label_index+1)]
        # This code UNHOTs the labels:
        # out = '|'.join([index_to_label.get(idx) for idx, val in enumerate(one_hot_tuple) if val == 1])
        return out

    def get_one_hot_labels(self, fname):
        """ Get the 1-hot encoded label array for the provided file """
        labels = self.get_labels(fname)
        one_hot_labels = self.one_hot_labels(labels)
        return torch.FloatTensor(one_hot_labels)

### Model Training Output Writer

In [5]:
class printer_writer:
    def __init__(self, output_folder_path):
        self.start_time = time.strftime('%Y%m%d-%Hh%Mm%Ss')
        
        self.outprefix = output_folder_path + '/' + self.start_time
        
        # Print Output File
        self.print_out_path = self.outprefix + '_print.txt'
        self.print_out_file = open(self.print_out_path, 'w', 1)
        
    def printw(self, string):
        print(string)
        try:
            self.print_out_file.write(string + "\n")
        except: # Ignore errors
            pass
        
    def save_checkpoint(self, epoch, model, optimizer, scheduler, val_error):
        model_out_path = self.outprefix + '_model_' + str(epoch+1) + '.tar'
        
        torch.save({
            'epoch': epoch+1,
            'state': model.state_dict(),
            'optimizer': optimizer,
            'scheduler': scheduler,
            'val_error': val_error
        }, model_out_path)
        
    def close(self):
        self.print_out_file.close()

### Model Training Procedure

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, outfolder = '/user/xrayproj/output/'):
    since = time.time()
    scribe = printer_writer(outfolder)

    for epoch in range(num_epochs):
        scribe.printw('Epoch {}/{}'.format(epoch, num_epochs - 1))
        scribe.printw('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0
            obs_counter = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # Store statistics (convert from autograd.Variable to float/int)
                loss_val = loss.data[0]
                correct_val = torch.sum( ((outputs.sigmoid()>0.5) == (labels>0.5)).long() ).data[0]
                
                running_loss += loss_val
                running_corrects += correct_val
                
                obs_counter += len(inputs)
                
                batch_loss = 1.0 * loss_val / len(inputs)
                batch_acc = 1.0 * correct_val / len(inputs)
                status = ' |~~ {}@{}  Loss: {:.6f} Acc: {:.4f}'.format(
                    phase, obs_counter, batch_loss, batch_acc)
                scribe.printw(status)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)
            scribe.printw('{}  Loss: {:.6f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # Store the model on disk
            if phase == 'val':
                scheduler.step(epoch_loss)
                if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                    scribe.save_checkpoint(epoch, model, optimizer, None, epoch_loss)
                else:
                    scribe.save_checkpoint(epoch, model, optimizer, scheduler, epoch_loss)

    time_elapsed = time.time() - since
    scribe.printw('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    scribe.close()

    return model

### Customized Binary Cross Entropy Loss Function

In [7]:
class BCEWithLogitsImbalanceWeightedLoss(Module):
    def __init__(self, class_weight=None, size_average=True):
        super(BCEWithLogitsImbalanceWeightedLoss, self).__init__()
        self.size_average = size_average

    def forward(self, input, target):
        return self.imbalance_weighted_bce_with_logit(input, target, size_average=self.size_average)
    
    def imbalance_weighted_bce_with_logit(self, input, target, size_average=True):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})".format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + ((-max_val).exp() + (-input - max_val).exp()).log()

        # Determine |P| and |N|
        positive_labels = target.sum()
        negative_labels = (1-target).sum()

        # Upweight the less common class (very often the 1s)
        beta_p = (positive_labels + negative_labels) / positive_labels
        beta_n = (positive_labels + negative_labels) / negative_labels

        # Adjust the losses accordingly
        loss_weight = target * beta_p + (1-target) * beta_n

        loss = loss * loss_weight

        if size_average:
            return loss.mean()
        else:
            return loss.sum()

### Define model types

In [15]:
def ResNetBase(base_size = 18, only_update_fc = True):
    """ ResNet 18 with only final FC layer updatable """
    m = None
    if base_size == 18:
        m = models.resnet18(pretrained=True)
    elif base_size == 34:
        m = models.resnet34(pretrained=True)
    elif base_size == 50:
        m = models.resnet50(pretrained=True)
    elif base_size == 101:
        m = models.resnet101(pretrained=True)
    elif base_size == 152:
        m = models.resnet152(pretrained=True)
    
    if only_update_fc:
        for param in m.parameters():
            param.requires_grad = False

    m.fc = nn.Linear(m.fc.in_features, len(img_data_train.labels))
    
    return m

#### mean ± std. dev. of 7 runs, 10000000 loops each

#### Time for __get_item__
```
%timeit img_data_train[3] # 30.8 ms ± 544 µs per loop
```

#### Breakdown for __get_item__
```
%timeit img_data_train.imgs[8] # 63 ns ± 0.0057 ns per loop
%timeit img_data_train.get_one_hot_labels('00011558_012.png') # 8.72 ms ± 9.44 µs per loop
%timeit img_data_train.loader('/user/images/images_006/00011558_012.png') # 14.1 ms ± 3.41 µs per loop
%timeit img_data_train.transform(img) # 3.17 ms ± 1.32 µs per loop
```

#### Breakdown for loader() from __get_item__
```
%timeit open('/user/images/images_006/00011558_012.png', 'rb') # 7.72 µs ± 13.4 ns per loop
%timeit Image.open(f) # 37.5 µs ± 2.25 µs per loop
%timeit img.convert('RGB') # 498 µs ± 149 ns per loop
```

# Setup and Begin Training Model

### Data Setup

In [16]:
nn_input_size = 224 #1024
batch_size = 64
pin_mem_setting = True
num_gpus = torch.cuda.device_count()
num_workers = 10

print("Number of GPU: {}".format(num_gpus))

Number of GPU: 1


In [17]:
img_transforms_train = transforms.Compose(
    [#transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [18]:
img_transforms_nontrain = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [19]:
img_data_train = XrayImageSet(image_root = '/user/images_processed/',
                              csv_file = '/user/img_details.csv',
                              transform = img_transforms_train,
                              target_transform = None)

img_data_train.imgs = [img for i, img in enumerate(img_data_train.imgs) if i % 10 >= 3]

In [20]:
img_data_val   = XrayImageSet(image_root = '/user/images_processed/',
                              csv_file = '/user/img_details.csv',
                              transform = img_transforms_nontrain,
                              target_transform = None)

img_data_val.imgs = [img for i, img in enumerate(img_data_val.imgs) if i % 10 in (1,2)]

In [21]:
img_data_test  = XrayImageSet(image_root = '/user/images_processed/',
                              csv_file = '/user/img_details.csv',
                              transform = img_transforms_nontrain,
                              target_transform = None)

img_data_test.imgs = [img for i, img in enumerate(img_data_test.imgs) if i % 10 == 0]

In [22]:
train_set = set(img_data_train.imgs)
val_set = set(img_data_val.imgs)
test_set = set(img_data_test.imgs)
assert len(train_set.intersection(val_set)) == 0
assert len(train_set.intersection(test_set)) == 0
assert len(val_set.intersection(test_set)) == 0

In [23]:
print("Training Set Size: {}".format(len(img_data_train)))
print("Validation Set Size: {}".format(len(img_data_val)))
print("Test Set Size: {}".format(len(img_data_test)))

Training Set Size: 78484
Validation Set Size: 22424
Test Set Size: 11212


In [24]:
img_loader_train = DataLoader(img_data_train,
                              batch_size = batch_size * num_gpus,
                              shuffle = True,
                              num_workers = num_workers,
                              pin_memory = pin_mem_setting)

img_loader_val   = DataLoader(img_data_val,
                              batch_size = batch_size * num_gpus,
                              shuffle = True,
                              num_workers = num_workers,
                              pin_memory = pin_mem_setting)

In [25]:
dataloaders = {
    'train': img_loader_train,
    'val': img_loader_val
}

In [33]:
model_base = ResNetBase(base_size = 18, only_update_fc = True)

In [34]:
model_ft = DataParallel(model_base).cuda()

### Setup learning rates and procedures

In [35]:
#criterion = BCEWithLogitsImbalanceWeightedLoss()
criterion_base = nn.BCEWithLogitsLoss()

optimizer_ft = SGD(model_ft.module.fc.parameters(), lr=0.01, momentum=0.9)
#optimizer_ft = SGD(model_ft.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
#lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
learning_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, patience=0)

In [36]:
criterion = criterion_base.cuda()

### Future code for allowing optimization of the base layer with a lower learning rate

```
ignored_params = list(map(id, model.fc.parameters()))
base_params = filter(lambda p: id(p) not in ignored_params,
                     model.parameters())

optimizer = torch.optim.SGD([
            {'params': base_params},
            {'params': model.fc.parameters(), 'lr': opt.lr}
        ], lr=opt.lr*0.1, momentum=0.9)
```

# Begin Training Network (Normal Cost)

In [ ]:
train_model(model_ft,
            criterion,
            optimizer_ft,
            learning_scheduler,
            num_epochs=10)

Epoch 0/9
----------
 |~~ train@64  Loss: 0.011634 Acc: 6.9219
 |~~ train@128  Loss: 0.010655 Acc: 7.9219
 |~~ train@192  Loss: 0.009005 Acc: 10.2656
 |~~ train@256  Loss: 0.007177 Acc: 12.5156
 |~~ train@320  Loss: 0.005410 Acc: 13.4219
 |~~ train@384  Loss: 0.004334 Acc: 13.3438
 |~~ train@448  Loss: 0.003355 Acc: 13.4062
 |~~ train@512  Loss: 0.003276 Acc: 13.2656
 |~~ train@576  Loss: 0.003737 Acc: 13.0312
 |~~ train@640  Loss: 0.003266 Acc: 13.1875
 |~~ train@704  Loss: 0.002882 Acc: 13.2969
 |~~ train@768  Loss: 0.003613 Acc: 13.1562
 |~~ train@832  Loss: 0.002868 Acc: 13.3281
 |~~ train@896  Loss: 0.002253 Acc: 13.5312
 |~~ train@960  Loss: 0.003345 Acc: 13.3125
 |~~ train@1024  Loss: 0.002522 Acc: 13.4844
 |~~ train@1088  Loss: 0.003177 Acc: 13.3438
 |~~ train@1152  Loss: 0.004283 Acc: 13.0781
 |~~ train@1216  Loss: 0.002363 Acc: 13.5312
 |~~ train@1280  Loss: 0.003883 Acc: 13.1406
 |~~ train@1344  Loss: 0.002413 Acc: 13.5156
 |~~ train@1408  Loss: 0.002887 Acc: 13.2969
 |~~ tr

 |~~ train@11776  Loss: 0.003176 Acc: 13.1719
 |~~ train@11840  Loss: 0.003065 Acc: 13.1875
 |~~ train@11904  Loss: 0.002752 Acc: 13.3281
 |~~ train@11968  Loss: 0.002826 Acc: 13.3438
 |~~ train@12032  Loss: 0.002832 Acc: 13.2031
 |~~ train@12096  Loss: 0.002732 Acc: 13.3594
 |~~ train@12160  Loss: 0.003303 Acc: 13.1250
 |~~ train@12224  Loss: 0.002940 Acc: 13.2344
 |~~ train@12288  Loss: 0.002836 Acc: 13.2812
 |~~ train@12352  Loss: 0.002915 Acc: 13.2188
 |~~ train@12416  Loss: 0.003329 Acc: 13.0938
 |~~ train@12480  Loss: 0.002755 Acc: 13.2969
 |~~ train@12544  Loss: 0.002221 Acc: 13.4531
 |~~ train@12608  Loss: 0.002919 Acc: 13.2344
 |~~ train@12672  Loss: 0.003227 Acc: 13.1875
 |~~ train@12736  Loss: 0.003162 Acc: 13.1094
 |~~ train@12800  Loss: 0.002673 Acc: 13.3125
 |~~ train@12864  Loss: 0.002903 Acc: 13.2500
 |~~ train@12928  Loss: 0.002525 Acc: 13.3594
 |~~ train@12992  Loss: 0.002238 Acc: 13.4688
 |~~ train@13056  Loss: 0.003073 Acc: 13.2344
 |~~ train@13120  Loss: 0.002783 A

 |~~ train@23296  Loss: 0.002679 Acc: 13.3125
 |~~ train@23360  Loss: 0.003396 Acc: 13.1250
 |~~ train@23424  Loss: 0.002786 Acc: 13.2500
 |~~ train@23488  Loss: 0.003905 Acc: 13.0156
 |~~ train@23552  Loss: 0.002662 Acc: 13.2812
 |~~ train@23616  Loss: 0.003110 Acc: 13.2031
 |~~ train@23680  Loss: 0.002407 Acc: 13.3906
 |~~ train@23744  Loss: 0.002552 Acc: 13.3281
 |~~ train@23808  Loss: 0.002014 Acc: 13.5469
 |~~ train@23872  Loss: 0.002823 Acc: 13.2500
 |~~ train@23936  Loss: 0.003279 Acc: 13.1719
 |~~ train@24000  Loss: 0.002694 Acc: 13.2969
 |~~ train@24064  Loss: 0.002676 Acc: 13.3906
 |~~ train@24128  Loss: 0.002488 Acc: 13.3750
 |~~ train@24192  Loss: 0.002430 Acc: 13.4531
 |~~ train@24256  Loss: 0.002403 Acc: 13.4219
 |~~ train@24320  Loss: 0.002413 Acc: 13.4062
 |~~ train@24384  Loss: 0.002811 Acc: 13.2812
 |~~ train@24448  Loss: 0.002920 Acc: 13.2344
 |~~ train@24512  Loss: 0.002944 Acc: 13.2500
 |~~ train@24576  Loss: 0.002297 Acc: 13.3750
 |~~ train@24640  Loss: 0.002313 A

 |~~ train@34752  Loss: 0.002070 Acc: 13.5312
 |~~ train@34816  Loss: 0.002962 Acc: 13.2188
 |~~ train@34880  Loss: 0.002740 Acc: 13.2969
 |~~ train@34944  Loss: 0.002272 Acc: 13.3906
 |~~ train@35008  Loss: 0.002660 Acc: 13.2812
 |~~ train@35072  Loss: 0.002186 Acc: 13.4844
 |~~ train@35136  Loss: 0.003058 Acc: 13.2344
 |~~ train@35200  Loss: 0.002460 Acc: 13.3438
 |~~ train@35264  Loss: 0.002845 Acc: 13.2188
 |~~ train@35328  Loss: 0.002775 Acc: 13.2031
 |~~ train@35392  Loss: 0.002507 Acc: 13.4062
 |~~ train@35456  Loss: 0.003053 Acc: 13.2031
 |~~ train@35520  Loss: 0.002842 Acc: 13.2812
 |~~ train@35584  Loss: 0.002843 Acc: 13.2812
 |~~ train@35648  Loss: 0.002518 Acc: 13.3281
 |~~ train@35712  Loss: 0.002796 Acc: 13.2031
 |~~ train@35776  Loss: 0.002914 Acc: 13.1875
 |~~ train@35840  Loss: 0.002330 Acc: 13.4531
 |~~ train@35904  Loss: 0.002830 Acc: 13.1719
 |~~ train@35968  Loss: 0.002275 Acc: 13.4844
 |~~ train@36032  Loss: 0.003402 Acc: 13.0156
 |~~ train@36096  Loss: 0.002603 A

 |~~ train@46208  Loss: 0.003442 Acc: 13.0625
 |~~ train@46272  Loss: 0.003063 Acc: 13.2031
 |~~ train@46336  Loss: 0.003308 Acc: 13.1094
 |~~ train@46400  Loss: 0.002494 Acc: 13.4375
 |~~ train@46464  Loss: 0.002983 Acc: 13.2031
 |~~ train@46528  Loss: 0.002407 Acc: 13.3438
 |~~ train@46592  Loss: 0.002821 Acc: 13.2812
 |~~ train@46656  Loss: 0.002755 Acc: 13.2812
 |~~ train@46720  Loss: 0.002084 Acc: 13.4844
 |~~ train@46784  Loss: 0.003080 Acc: 13.2188
 |~~ train@46848  Loss: 0.002419 Acc: 13.3906
 |~~ train@46912  Loss: 0.002331 Acc: 13.4062
 |~~ train@46976  Loss: 0.002897 Acc: 13.2188
 |~~ train@47040  Loss: 0.002764 Acc: 13.2656
 |~~ train@47104  Loss: 0.002119 Acc: 13.4844
 |~~ train@47168  Loss: 0.002560 Acc: 13.3594
 |~~ train@47232  Loss: 0.002174 Acc: 13.5156
 |~~ train@47296  Loss: 0.002848 Acc: 13.2500
 |~~ train@47360  Loss: 0.002955 Acc: 13.2188
 |~~ train@47424  Loss: 0.002848 Acc: 13.2188
 |~~ train@47488  Loss: 0.002451 Acc: 13.3906
 |~~ train@47552  Loss: 0.002156 A

 |~~ train@57728  Loss: 0.002769 Acc: 13.2188
 |~~ train@57792  Loss: 0.002607 Acc: 13.2031
 |~~ train@57856  Loss: 0.002971 Acc: 13.2500
 |~~ train@57920  Loss: 0.002846 Acc: 13.2500
 |~~ train@57984  Loss: 0.002744 Acc: 13.2969
 |~~ train@58048  Loss: 0.002958 Acc: 13.2031
 |~~ train@58112  Loss: 0.003361 Acc: 13.0781
 |~~ train@58176  Loss: 0.002194 Acc: 13.4531
 |~~ train@58240  Loss: 0.002592 Acc: 13.3438
 |~~ train@58304  Loss: 0.002927 Acc: 13.2344
 |~~ train@58368  Loss: 0.002907 Acc: 13.2188
 |~~ train@58432  Loss: 0.002797 Acc: 13.3438
 |~~ train@58496  Loss: 0.002835 Acc: 13.2344
 |~~ train@58560  Loss: 0.003050 Acc: 13.1250
 |~~ train@58624  Loss: 0.003145 Acc: 13.1562
 |~~ train@58688  Loss: 0.002638 Acc: 13.2969
 |~~ train@58752  Loss: 0.002500 Acc: 13.3281
 |~~ train@58816  Loss: 0.002397 Acc: 13.3906
 |~~ train@58880  Loss: 0.002370 Acc: 13.3750
 |~~ train@58944  Loss: 0.002449 Acc: 13.3281
 |~~ train@59008  Loss: 0.002793 Acc: 13.2812
 |~~ train@59072  Loss: 0.003323 A

 |~~ train@69248  Loss: 0.002459 Acc: 13.3125
 |~~ train@69312  Loss: 0.002961 Acc: 13.2031
 |~~ train@69376  Loss: 0.002897 Acc: 13.2031
 |~~ train@69440  Loss: 0.003031 Acc: 13.1875
 |~~ train@69504  Loss: 0.002810 Acc: 13.2812
 |~~ train@69568  Loss: 0.002474 Acc: 13.3281
 |~~ train@69632  Loss: 0.002386 Acc: 13.3750
 |~~ train@69696  Loss: 0.002955 Acc: 13.1094
 |~~ train@69760  Loss: 0.002445 Acc: 13.3906
 |~~ train@69824  Loss: 0.002939 Acc: 13.2188
 |~~ train@69888  Loss: 0.002832 Acc: 13.2812
 |~~ train@69952  Loss: 0.003327 Acc: 13.0469
 |~~ train@70016  Loss: 0.003065 Acc: 13.1250
 |~~ train@70080  Loss: 0.002476 Acc: 13.2969
 |~~ train@70144  Loss: 0.002855 Acc: 13.1250
 |~~ train@70208  Loss: 0.002303 Acc: 13.3438
 |~~ train@70272  Loss: 0.002641 Acc: 13.2500
 |~~ train@70336  Loss: 0.002994 Acc: 13.1094
 |~~ train@70400  Loss: 0.002655 Acc: 13.2812
 |~~ train@70464  Loss: 0.002868 Acc: 13.1719
 |~~ train@70528  Loss: 0.002871 Acc: 13.2656
 |~~ train@70592  Loss: 0.002936 A

 |~~ val@2240  Loss: 0.003036 Acc: 13.2031
 |~~ val@2304  Loss: 0.002827 Acc: 13.2812
 |~~ val@2368  Loss: 0.003150 Acc: 13.0938
 |~~ val@2432  Loss: 0.002489 Acc: 13.3906
 |~~ val@2496  Loss: 0.002255 Acc: 13.4219
 |~~ val@2560  Loss: 0.002170 Acc: 13.4688
 |~~ val@2624  Loss: 0.003219 Acc: 13.0625
 |~~ val@2688  Loss: 0.002803 Acc: 13.1875
 |~~ val@2752  Loss: 0.002508 Acc: 13.3125
 |~~ val@2816  Loss: 0.002630 Acc: 13.2500
 |~~ val@2880  Loss: 0.002987 Acc: 13.2031
 |~~ val@2944  Loss: 0.002489 Acc: 13.3438
 |~~ val@3008  Loss: 0.002990 Acc: 13.1875
 |~~ val@3072  Loss: 0.002601 Acc: 13.3750
 |~~ val@3136  Loss: 0.003126 Acc: 13.1406
 |~~ val@3200  Loss: 0.002609 Acc: 13.2656
 |~~ val@3264  Loss: 0.002609 Acc: 13.2344
 |~~ val@3328  Loss: 0.002784 Acc: 13.3281
 |~~ val@3392  Loss: 0.002730 Acc: 13.3125
 |~~ val@3456  Loss: 0.002506 Acc: 13.3750
 |~~ val@3520  Loss: 0.003262 Acc: 13.0781
 |~~ val@3584  Loss: 0.003136 Acc: 13.1875
 |~~ val@3648  Loss: 0.002262 Acc: 13.4062
 |~~ val@37

### Save model results to S3

aws s3 cp ResNet18PlusFlexibleFC_Epoch9.tar s3://bdh-xrayproj-modelparameters/

import boto3

s3 = boto3.client('s3')
s3.list_buckets()

S3 Commands: http://docs.aws.amazon.com/cli/latest/userguide/using-s3-commands.html

Boto3 QuickStart: http://boto3.readthedocs.io/en/latest/guide/quickstart.html

Key Management: https://aws.amazon.com/blogs/security/a-safer-way-to-distribute-aws-credentials-to-ec2/

AWS IAM Rules: http://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_use_switch-role-api.html